In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier

/home/boada/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/boada/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Load data

In [2]:
inspecs= pd.read_csv('./data/inspecs_2016_map.csv')

In [3]:
inspecs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27666 entries, 0 to 27665
Data columns (total 20 columns):
camis                    27666 non-null int64
dba                      27666 non-null object
boro                     27666 non-null object
zipcode                  27666 non-null float64
cuisine_description      27666 non-null object
inspection_date          27666 non-null object
action                   27666 non-null object
violation_code           27343 non-null object
violation_description    27321 non-null object
critical_flag            27666 non-null object
score                    26295 non-null float64
grade                    14021 non-null object
inspection_type          27666 non-null object
latitude                 27666 non-null float64
longitude                27666 non-null float64
weekday                  27666 non-null int64
311_score                27666 non-null float64
nypd_score               27666 non-null float64
boro_enc                 27666 non-null f

### Chose the columns we are going to work with

In [4]:
train_cols = ['weekday', '311_score', 'nypd_score', 'zipcode', 'boro_enc']
target_cols = ['crit_flag_enc']



In [5]:
data = inspecs[train_cols].values
target = inspecs[target_cols].values

X_train, X_test, y_train, y_test = train_test_split(data, target,
                                                    train_size=0.75, test_size=0.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape



((20749, 5), (6917, 5), (20749, 1), (6917, 1))

In [6]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2)
tpot.fit(X_train, y_train)
tpot.score(X_test, y_test)

/home/boada/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



2.35 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.1, min_samples_leaf=17, min_samples_split=11, n_estimators=100)
0.5587682521324273


/home/boada/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
tpot.export('test.py')

In [9]:
tpot.get_params()

{'config_dict': None,
 'crossover_rate': 0.1,
 'cv': 5,
 'disable_update_check': False,
 'early_stop': None,
 'generations': 100,
 'max_eval_time_mins': 5,
 'max_time_mins': 2,
 'memory': None,
 'mutation_rate': 0.9,
 'n_jobs': 1,
 'offspring_size': None,
 'periodic_checkpoint_folder': None,
 'population_size': 100,
 'random_state': None,
 'scoring': None,
 'subsample': 1.0,
 'template': None,
 'use_dask': False,
 'verbosity': 2,
 'warm_start': False}

In [10]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train)

/home/boada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [12]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [15]:
feature_list = train_cols


# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
for pair in feature_importances: print('Variable: {:20} Importance: {}'.format(*pair))

Variable: zipcode              Importance: 0.3
Variable: nypd_score           Importance: 0.28
Variable: 311_score            Importance: 0.25
Variable: weekday              Importance: 0.15
Variable: boro_enc             Importance: 0.02


In [16]:
from sklearn.metrics import confusion_matrix